In [1]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=NLJ1C1S42IC6LW7YB4GQFOZXZJGJSC
%env CLEARML_API_SECRET_KEY=qVctArALlJ5gxnUmmftsdLFO5mGdAhBtnw0IWcvF-5Hbhan-h4wEcHoTMjGCj19PKM4

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=NLJ1C1S42IC6LW7YB4GQFOZXZJGJSC
env: CLEARML_API_SECRET_KEY=qVctArALlJ5gxnUmmftsdLFO5mGdAhBtnw0IWcvF-5Hbhan-h4wEcHoTMjGCj19PKM4


In [4]:
# Добавьте в начало файла установку ClearML и импорт
# В Jupyter Notebook сначала выполните:
# !pip install clearml

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import DateOffset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import pickle
import joblib
# Добавьте этот импорт
from PIL import Image

# ========== CLEARML ИНИЦИАЛИЗАЦИЯ ==========
from clearml import Task

# Создаем задачу ClearML
task = Task.init(
    project_name="Customer_Purchase_Prediction",
    task_name="RandomForest_Prediction_Experiment",
    task_type=Task.TaskTypes.training,
    reuse_last_task_id=False
)

# Логируем информацию о задаче
print("ClearML Task создана. ID:", task.id)

# ========== ЛОГИРОВАНИЕ ИСХОДНЫХ ДАННЫХ КАК АРТЕФАКТ ==========
import os
import hashlib
from datetime import datetime

# Путь к исходному файлу данных
source_file_path = 'C:/Users/maris/OneDrive/Рабочий стол/transaction.csv'

# Создаем копию исходного файла с метаданными
def prepare_data_artifact(file_path):
    """
    Подготавливает и логирует исходные данные как артефакт ClearML
    """
    try:
        # Проверяем существование файла
        if not os.path.exists(file_path):
            print(f"Внимание: Файл {file_path} не найден!")
            return None
        
        # Читаем файл для получения информации о нем
        file_size = os.path.getsize(file_path)
        file_name = os.path.basename(file_path)
        file_extension = os.path.splitext(file_path)[1]
        
        # Вычисляем хэш файла для проверки целостности
        def calculate_file_hash(filepath):
            hash_md5 = hashlib.md5()
            with open(filepath, "rb") as f:
                for chunk in iter(lambda: f.read(4096), b""):
                    hash_md5.update(chunk)
            return hash_md5.hexdigest()
        
        file_hash = calculate_file_hash(file_path)
        
        # Создаем метаданные файла
        metadata = {
            'filename': file_name,
            'file_path': file_path,
            'file_size_bytes': file_size,
            'file_size_mb': round(file_size / (1024 * 1024), 2),
            'file_extension': file_extension,
            'file_hash_md5': file_hash,
            'timestamp': datetime.now().isoformat(),
            'rows_count': None,
            'columns_count': None,
            'columns': None
        }
        
        # Читаем первые несколько строк для предварительного анализа
        try:
            df_preview = pd.read_csv(file_path, nrows=5)
            metadata['rows_count_preview'] = len(df_preview)
            metadata['columns_count'] = len(df_preview.columns)
            metadata['columns'] = list(df_preview.columns)
            
            # Сохраняем предпросмотр в отдельный файл
            preview_file = 'data_preview.csv'
            df_preview.to_csv(preview_file, index=False)
            
        except Exception as e:
            print(f"Предварительный анализ файла не удался: {e}")
        
        # Логируем метаданные
        task.get_logger().report_text(f"Исходный файл данных: {file_name}")
        task.get_logger().report_text(f"Размер файла: {metadata['file_size_mb']} MB")
        task.get_logger().report_text(f"MD5 хэш: {file_hash}")
        
        # Сохраняем метаданные в файл
        import json
        metadata_file = 'data_metadata.json'
        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        
        # Логируем файл как артефакт
        print(f"\nЛогирование исходных данных...")
        
        # Способ 1: Прикрепляем оригинальный файл (если он не слишком большой)
        if file_size < 100 * 1024 * 1024:  # Если файл меньше 100MB
            task.upload_artifact(
                name='raw_source_data',
                artifact_object=file_path,
                metadata=metadata
            )
            print(f"✓ Оригинальный файл прикреплен как артефакт 'raw_source_data'")
        else:
            print(f"⚠ Файл слишком большой ({metadata['file_size_mb']} MB), создаем сэмпл")
            # Создаем сэмпл данных (первые 10,000 строк)
            df_sample = pd.read_csv(file_path, nrows=10000)
            sample_file = 'data_sample_10000_rows.csv'
            df_sample.to_csv(sample_file, index=False)
            
            task.upload_artifact(
                name='data_sample',
                artifact_object=sample_file,
                metadata={**metadata, 'sample_rows': 10000}
            )
            print(f"✓ Сэмпл данных (10,000 строк) прикреплен как артефакт 'data_sample'")
        
        # Способ 2: Прикрепляем метаданные и предпросмотр
        task.upload_artifact(
            name='data_metadata',
            artifact_object=metadata_file,
            metadata={'description': 'Метаданные исходного файла данных'}
        )
        
        if 'preview_file' in locals():
            task.upload_artifact(
                name='data_preview',
                artifact_object=preview_file,
                metadata={'description': 'Предпросмотр первых 5 строк данных'}
            )
        
        # Сохраняем путь к файлу в конфигурации задачи
        task.connect_configuration(
            name='data_source',
            configuration={
                'source_file': file_path,
                'file_hash': file_hash,
                'upload_timestamp': metadata['timestamp']
            }
        )
        
        print("✓ Исходные данные успешно залогированы в ClearML")
        print(f"  Хэш файла: {file_hash}")
        print(f"  Размер: {metadata['file_size_mb']} MB")
        
        return metadata
        
    except Exception as e:
        print(f"Ошибка при логировании исходных данных: {e}")
        return None

# Вызываем функцию логирования данных
data_metadata = prepare_data_artifact(source_file_path)

# ========== ПРОДОЛЖАЕМ С ЗАГРУЗКОЙ ДАННЫХ ==========

# ========== ВАШ СУЩЕСТВУЮЩИЙ КОД ==========
# Загружаем таблицу из файла в переменную df (data frame) и переименовываем колонки
df = pd.read_csv('C:/Users/maris/OneDrive/Рабочий стол/transaction.csv')

rename_dict = {
    'clientID': 'client',
    'trDte': 'visit_date', 
    'itemGroup': 'item_group'
}

df = df.rename(columns=rename_dict)
# Преобразование даты (уже с новым именем)
df['visit_date'] = pd.to_datetime(df['visit_date'], format='%d.%m.%Y')

# ========== ОБНОВЛЕННАЯ ФУНКЦИЯ С ЛОГИРОВАНИЕМ ==========
def aggregate_client_daily_items(df):
    """
    Функция группирует покупки по клиенту, дате, товару и группе.
    Суммирует количество и сумму. Возвращает отсортированную таблицу.
    """
    # Логируем начало обработки данных
    task.get_logger().report_text("Начинаем агрегацию данных клиентов")
    
    # Группируем строки по: клиенту, дате, товару, группе товара
    grouped = df.groupby(
        ['client', 'visit_date', 'item', 'item_group'],  # по этим колонкам объединяем
        as_index=False  # чтобы client и другие не стали индексом
    ).agg({
        'quantity': 'sum',  # складываем количество
        'amount': 'sum'     # складываем сумму
    })
    
    # Переименовываем колонку visit_date в visit_date, как требуется
    grouped = grouped.rename(columns={'visit_date': 'visit_date'})
    
    # Сортируем результат по client, потом по дате, потом по товару
    result = grouped.sort_values(by=['client', 'visit_date', 'item'])
    
    # Сбрасываем номера строк, чтобы они шли по порядку (0, 1, 2...)
    result = result.reset_index(drop=True)
    
    # Логируем размер результата
    task.get_logger().report_text(f"Агрегация завершена. Размер данных: {result.shape}")
    
    # Возвращаем итоговую таблицу
    return result

result_df = aggregate_client_daily_items(df)

# ========== ДОПОЛНИТЕЛЬНЫЕ ФУНКЦИИ ==========
def calculate_client_profile_at_date(visits_df, observation_end_date):
    """
    Строит профиль клиента на дату observation_end_date (исключая её).
    Нет утечки будущего — только данные СТРОГО ДО этой даты.
    """
    
    # Логируем параметры функции
    task.get_logger().report_text(f"Строим профиль клиента на дату: {observation_end_date}")
    
    # ... (ваш существующий код функции calculate_client_profile_at_date)
    obs_date = pd.to_datetime(observation_end_date)
    
    df = visits_df.copy()
    df['visit_date'] = pd.to_datetime(df['visit_date'])
    
    # Оставляем только визиты СТРОГО ДО observation_end_date
    filtered_df = df[df['visit_date'] < obs_date]
    
    if filtered_df.empty:
        return pd.DataFrame(columns=[
            'client', 'Recency', 'Frequency', 'Monetary', 'last_visit_date',
            'total_quantity', 'avg_check', 'total_unique_items',
            'avg_items_per_visit', 'weekend_visits', 'amount_last_visit',
            'share_3_month'
        ])
    
    profiles = []
    
    for client_id, group in filtered_df.groupby('client'):
        # ... (ваш существующий код обработки клиентов)
        group = group.sort_values(by='visit_date')
        visit_dates = group['visit_date'].drop_duplicates().sort_values()
        n_visits = len(visit_dates)
        total_monetary = group['amount'].sum()
        total_quantity = group['quantity'].sum()
        last_visit = visit_dates.max()
        recency_days = (obs_date - last_visit).days
        frequency = n_visits
        monetary = total_monetary
        last_visit_date = last_visit.strftime('%Y-%m-%d')
        avg_check = monetary / frequency if frequency > 0 else 0
        total_unique_items = group['item'].nunique()
        avg_items_per_visit = total_quantity / frequency if frequency > 0 else 0
        
        visit_days = visit_dates.dt.weekday
        weekend_count = visit_days.isin([5, 6]).sum()
        weekend_visits = weekend_count / frequency if frequency > 0 else 0
        
        last_visit_date_val = visit_dates.max()
        last_visit_rows = group[group['visit_date'] == last_visit_date_val]
        amount_last_visit_value = last_visit_rows['amount'].sum()
        amount_last_visit = (
            amount_last_visit_value / total_monetary
            if total_monetary > 0 else 0
        )
        
        three_months_ago = obs_date - pd.Timedelta(days=90)
        recent_visits = group[group['visit_date'] >= three_months_ago]
        amount_last_3m = recent_visits['amount'].sum()
        share_3_month = amount_last_3m / total_monetary if total_monetary > 0 else 0
        
        profile = {
            'client': client_id,
            'Recency': recency_days,
            'Frequency': frequency,
            'Monetary': monetary,
            'last_visit_date': last_visit_date,
            'total_quantity': total_quantity,
            'avg_check': round(avg_check, 2),
            'total_unique_items': total_unique_items,
            'avg_items_per_visit': round(avg_items_per_visit, 2),
            'weekend_visits': round(weekend_visits, 3),
            'amount_last_visit': round(amount_last_visit, 3),
            'share_3_month': round(share_3_month, 3)
        }
        
        profiles.append(profile)
    
    result_df = pd.DataFrame(profiles)
    result_df = result_df.sort_values(by='client').reset_index(drop=True)
    
    task.get_logger().report_text(f"Профиль построен. Клиентов: {len(result_df)}")
    
    return result_df

result_profile = calculate_client_profile_at_date(
    visits_df=result_df, 
    observation_end_date='2019-09-01'
)

# ========== ФУНКЦИЯ ДЛЯ СОЗДАНИЯ ТРЕНИРОВОЧНОЙ ВЫБОРКИ ==========
def mark_events(visits_df, result_start_date, result_end_date):
    """
    Размечает, какие клиенты посетили магазин в указанном периоде.
    """
    # ... (ваш существующий код функции mark_events)
    start_date = pd.to_datetime(result_start_date)
    end_date = pd.to_datetime(result_end_date)
    
    df = visits_df.copy()
    df['visit_date'] = pd.to_datetime(df['visit_date'])
    
    all_clients = df['client'].unique()
    result = pd.DataFrame({'client': all_clients})
    result['event'] = False
    
    mask = (df['visit_date'] >= start_date) & (df['visit_date'] < end_date)
    filtered_visits = df[mask]
    clients_with_visits = filtered_visits['client'].unique()
    
    result.loc[result['client'].isin(clients_with_visits), 'event'] = True
    result = result.sort_values(by='client').reset_index(drop=True)
    
    task.get_logger().report_text(f"Разметка событий завершена. Клиентов с событиями: {len(clients_with_visits)}")
    
    return result

events = mark_events(
    visits_df=result_df,
    result_start_date='2019-09-01',
    result_end_date='2019-10-01'
)

# ========== ФУНКЦИЯ СОЗДАНИЯ ТРЕНИРОВОЧНОЙ ВЫБОРКИ ==========
def create_training_sample(profile_df, events_df):
    """
    Создаёт обучающую выборку, соединяя профили клиентов и разметку события.
    """
    # Логируем создание выборки
    task.get_logger().report_text("Создаем обучающую выборку")
    
    merged = pd.merge(profile_df, events_df, on='client', how='inner')
    
    if 'event' not in merged.columns:
        raise ValueError("После объединения нет колонки 'event'")
    
    if merged['event'].isnull().any():
        raise ValueError("В колонке 'event' есть пропущенные значения")
    
    columns_to_drop = ['last_visit_date']
    columns_to_drop = [col for col in columns_to_drop if col in merged.columns]
    merged = merged.drop(columns=columns_to_drop)
    
    event_values = merged['event'].unique()
    if not (True in event_values and False in event_values):
        raise ValueError("В выборке нет обоих классов (True и False). Модель не сможет обучиться.")
    
    merged = merged.sort_values(by='client').reset_index(drop=True)
    
    # Логируем информацию о выборке
    task.get_logger().report_text(f"Размер выборки: {merged.shape}")
    task.get_logger().report_text(f"Распределение классов: {merged['event'].value_counts().to_dict()}")
    
    return merged

sample = create_training_sample(result_profile, events)

# ========== ПОДГОТОВКА ДАННЫХ ДЛЯ МОДЕЛИ ==========
# Разделяем данные на признаки и целевую переменную
X = sample.drop(['client', 'event'], axis=1)
y = sample['event']

# Разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Логируем размеры выборок
task.get_logger().report_text(f"Тренировочная выборка: {X_train.shape}")
task.get_logger().report_text(f"Тестовая выборка: {X_test.shape}")

# ========== ОБУЧЕНИЕ МОДЕЛИ ==========
task.get_logger().report_text("Начинаем обучение модели RandomForest")

# Создаем и обучаем модель
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)

model.fit(X_train, y_train)

# ========== ПРЕДСКАЗАНИЯ И МЕТРИКИ ==========
# Делаем предсказания
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Вычисляем метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# ========== ЛОГИРОВАНИЕ МЕТРИК В CLEARML ==========
task.get_logger().report_scalar("Accuracy", "Test", accuracy, iteration=0)
task.get_logger().report_scalar("Precision", "Test", precision, iteration=0)
task.get_logger().report_scalar("Recall", "Test", recall, iteration=0)
task.get_logger().report_scalar("F1-Score", "Test", f1, iteration=0)
task.get_logger().report_scalar("ROC-AUC", "Test", roc_auc, iteration=0)

# Выводим метрики в консоль
print("\n" + "="*50)
print("МЕТРИКИ КАЧЕСТВА МОДЕЛИ:")
print("="*50)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print("="*50)

# ========== СОХРАНЕНИЕ МОДЕЛИ ==========
# Сохраняем модель в формате pickle
model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

# Прикрепляем модель как артефакт к задаче ClearML
task.upload_artifact(name='trained_model', artifact_object=model_filename)

# Также сохраняем с помощью joblib (альтернатива)
joblib.dump(model, 'random_forest_model.joblib')
task.upload_artifact(name='trained_model_joblib', artifact_object='random_forest_model.joblib')

print(f"Модель сохранена как: {model_filename}")
print(f"Модель прикреплена как артефакт к задаче ClearML")

# ========== ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ ==========
# Логируем важность признаков
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 важнейших признаков:")
print(feature_importance.head(10))

# Сохраняем важность признаков как артефакт
feature_importance.to_csv('feature_importance.csv', index=False)
task.upload_artifact(name='feature_importance', artifact_object='feature_importance.csv')

# ========== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ==========
# Сохраняем результаты предсказаний
results_df = X_test.copy()
results_df['actual'] = y_test.values
results_df['predicted'] = y_pred
results_df['predicted_proba'] = y_pred_proba

results_df.to_csv('prediction_results.csv', index=False)
task.upload_artifact(name='prediction_results', artifact_object='prediction_results.csv')

# ========== ВИЗУАЛИЗАЦИЯ И ЛОГИРОВАНИЕ ==========
# Создаем confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Сохраняем график
plt.tight_layout()
confusion_matrix_path = 'confusion_matrix.png'
plt.savefig(confusion_matrix_path, dpi=100)
plt.close()

# Загружаем изображение и логируем его
try:
    from PIL import Image
    img = Image.open(confusion_matrix_path)
    
    # Логируем изображение в ClearML
    if task:
        task.get_logger().report_image(
            title="Confusion Matrix",
            series="Test",
            image=img
        )
    print(f"Confusion Matrix сохранен как: {confusion_matrix_path}")
except Exception as e:
    print(f"Ошибка при сохранении изображения: {e}")

# Также создаем график важности признаков
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Важность признака')
plt.title('Топ-15 важнейших признаков')
plt.gca().invert_yaxis()
plt.tight_layout()

feature_importance_path = 'feature_importance.png'
plt.savefig(feature_importance_path, dpi=100)
plt.close()

# Логируем график важности признаков
try:
    img2 = Image.open(feature_importance_path)
    if task:
        task.get_logger().report_image(
            title="Feature Importance",
            series="Analysis",
            image=img2
        )
    print(f"Feature Importance сохранен как: {feature_importance_path}")
except Exception as e:
    print(f"Ошибка при сохранении графика важности признаков: {e}")

# Создаем ROC-AUC кривую (опционально)
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC кривая (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()

roc_curve_path = 'roc_curve.png'
plt.savefig(roc_curve_path, dpi=100)
plt.close()

# Логируем ROC кривую
try:
    img3 = Image.open(roc_curve_path)
    if task:
        task.get_logger().report_image(
            title="ROC Curve",
            series="Test",
            image=img3
        )
    print(f"ROC Curve сохранена как: {roc_curve_path}")
except Exception as e:
    print(f"Ошибка при сохранении ROC кривой: {e}")

print("\n" + "="*50)
print("ЭКСПЕРИМЕНТ УСПЕШНО ЗАВЕРШЕН")
if task:
    print(f"ID задачи ClearML: {task.id}")
    print(f"Ссылка на эксперимент: {task.get_output_log_web_page()}")
print("="*50)

# Создаем финальный отчет
print("\nФИНАЛЬНЫЙ ОТЧЕТ:")
print(f"Лучшая метрика (ROC-AUC): {roc_auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print(f"\nМодель сохранена как: {model_filename}")
print(f"Артефакты сохранены в текущей директории")

# Завершаем задачу
if task:
    task.close()
    print("\nЗадача ClearML закрыта успешно")

Can't get url information for git repo in C:\Users\maris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages
Can't get branch information for git repo in C:\Users\maris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages


ClearML Task: created new task id=2b361086488e4955aa2ef6fc221b8945


Can't get commit information for git repo in C:\Users\maris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages
Can't get diff information for git repo in C:\Users\maris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages


ClearML results page: https://app.clear.ml/projects/37329c57f7424fb081d789b1b4239e5f/experiments/2b361086488e4955aa2ef6fc221b8945/output/log
ClearML Task создана. ID: 2b361086488e4955aa2ef6fc221b8945
Исходный файл данных: transaction.csv
Размер файла: 82.91 MB
MD5 хэш: 5e35dca941603ad2709b18c7373889a8

Логирование исходных данных...
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
✓ Оригинальный файл прикреплен как артефакт 'raw_source_data'
✓ Исходные данные успешно залогированы в ClearML
  Хэш файла: 5e35dca941603ad2709b18c7373889a8
  Размер: 82.91 MB
Начинаем агрегацию данных клиентов
Агрегация завершена. Размер данных: (1003083, 6)
Строим профиль клиента на дату: 2019-09-01
Профиль построен. Клиентов: 39906
Разметка событий завершена. Клиентов с событиями: 8821
Создаем обучающую выборку
Размер выборки: (39906, 12)
Распределение классов: {False: 32375, True: 7531}
Тренировочная выборка: (31924, 10)
Тестовая выборка: (7982, 10)
Начинаем обучени

C:\Users\maris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\clearml\utilities\plotlympl\renderer.py:209: UserWarning:

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.



...

cannot write mode RGBA as JPEG
cannot write mode RGBA as JPEG
cannot write mode RGBA as JPEG


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
ClearML Monitor: Reporting detected, reverting back to iteration based reporting


Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)': /v2.23/events.add_batch
Retrying (Retry(total=236, connect=240, read=236, redirect=240, status=240)) after connection broken by 'ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)': /v2.23/events.add_batch
Retrying (Retry(total=235, connect=240, read=235, redirect=240, status=240)) after connection broken by 'ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)': /v2.23/events.add_batch
Retrying (Retry(total=234, connect=240, read=234, redirect=240, status=240)) after connection broken by 'ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)': /v2.23/events.add_batch
Retrying (Retry(total=23


Задача ClearML закрыта успешно


Retrying (Retry(total=234, connect=240, read=234, redirect=240, status=240)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.clear.ml', port=443): Read timed out. (read timeout=10.0)")': /v2.23/tasks.ping
